# Use basic web scraping to pull a list of detailed phenotypes for each CHDgene gene

In [23]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import httplib2 as http
import json
import numpy as np
import pandas as pd
import re
import httplib2 as http
import json
import os
import glob
import warnings
warnings.filterwarnings(action='once')

# Import a list of all human TFs

In [76]:
tf_df=pd.read_csv('http://humantfs.ccbr.utoronto.ca/download/v_1.01/DatabaseExtract_v_1.01.csv',index_col='HGNC symbol') # import a df of all human TFs
tf_df=tf_df.drop(['Unnamed: 0'],axis=1) # tidy
tf_list=tf_df.index.tolist() # convert gene symbol to list

# Import CHDgene table

In [77]:
df=pd.read_csv('/home/jovyan/notebooks/suspension/scanpy_clustering/marker_lists/CHDgene/chdgene_table.csv') # downloaded from https://chdgene.victorchang.edu.au/
df=df.set_index('Gene')
df=df.rename_axis('gene')



df['TF']=0
df['TF'][df.index.isin(tf_list)]=1 # add a column indicating whether TF or not
df['source']='CHDgene'

CHDgene=df
CHDgene.to_csv('/home/jovyan/data/monogenic_disease/CHDgene_detailed_phenotype.csv')
CHDgene.head(3)

/tmp/ipykernel_5094/1753222255.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TF'][df.index.isin(tf_list)]=1 # add a column indicating whether TF or not


,CHD classification,Extra cardiac phenotype,Inheritance mode,Ranking,Supporting References,TF,source
gene,,,,,,,
ABL1,"ASD,VSD",Yes,"AD,DN",5,2,1,CHDgene
ACTC1,"ASD,VSD",Yes,AD,5,3,0,CHDgene
ACVR1,"ASD,AVSD and variants,Malformation of outflow ...",No,"AD,AR",5,2,0,CHDgene


In [6]:
# Use HGNC API to get entrez id for gene symbols

list_of_gene_symbols=chd.index.tolist()

list_of_gene_entrez_ids=[]

for symbol in list_of_gene_symbols:
    try:
        from urlparse import urlparse
    except ImportError:
        from urllib.parse import urlparse

    headers = {
      'Accept': 'application/json',
    }

    uri = 'https://rest.genenames.org'
    path = f'/fetch/symbol/{symbol}'

    target = urlparse(uri+path)
    method = 'GET'
    body = ''

    h = http.Http()

    response, content = h.request(
      target.geturl(),
      method,
      body,
      headers)

    if response['status'] == '200':
      # assume that content is a json reply
      # parse content with the json module 
      data = json.loads(content)
      list_of_gene_entrez_ids.append(data['response']['docs'][0]['entrez_id'])
    else:
      print( 'Error detected: ' + response['status'])

In [7]:
# Use Beautifulsoup webscraping to grab traits for each gene (each webpage is accessed using entrez id)

list_of_bullet_points_for_each_gene=[]

for entrez_id in list_of_gene_entrez_ids:
    url = f'https://chdgene.victorchang.edu.au/gene/{entrez_id}'
    html_text = requests.get(url).text
    soup = BeautifulSoup(html_text, 'html.parser')
    
    bullet_points_for_this_gene=[]

    for ultag in soup.find_all('ul', {'class': 'pl-4 mb-4'}):
        for litag in ultag.find_all('li'):
            bullet_points_for_this_gene.append(litag.text)
    
    bullet_points_for_this_gene_modified=[x for x in bullet_points_for_this_gene if not x.startswith('\n')] # remove a different type of bullet point on the same page
    
    list_of_bullet_points_for_each_gene.append(bullet_points_for_this_gene_modified)

In [8]:
output={
    'gene_symbols':list_of_gene_symbols,
    'gene_entrez_ids':list_of_gene_entrez_ids,
    'detailed_CHD_phenotype':list_of_bullet_points_for_each_gene
}

In [78]:
out=pd.DataFrame(output)
out=out.set_index('gene_symbols')
out

,gene_entrez_ids,detailed_CHD_phenotype
gene_symbols,,
ABL1,25,"[Atrial septal defect, Ventricular septal defect]"
ACTC1,70,"[Atrial septal defect, Ventricular septal defect]"
ACVR1,90,"[Atrial septal defect, Atrioventricular septal..."
ACVR2B,93,"[Atrial septal defect, Ventricular septal defe..."
ADAMTS10,81794,"[Ventricular septal defect, Aortic stenosis, P..."
...,...,...
UBR1,197131,"[Atrial septal defect, Ventricular septal defe..."
WASHC5,9897,"[Atrial septal defect, Ventricular septal defe..."
ZEB2,9839,"[Atrial septal defect, Ventricular septal defe..."


In [79]:
#join cols
CHDgene=CHDgene.join(out)
CHDgene

,CHD classification,Extra cardiac phenotype,Inheritance mode,Ranking,Supporting References,TF,source,gene_entrez_ids,detailed_CHD_phenotype
gene,,,,,,,,,
ABL1,"ASD,VSD",Yes,"AD,DN",5,2,1,CHDgene,25,"[Atrial septal defect, Ventricular septal defect]"
ACTC1,"ASD,VSD",Yes,AD,5,3,0,CHDgene,70,"[Atrial septal defect, Ventricular septal defect]"
ACVR1,"ASD,AVSD and variants,Malformation of outflow ...",No,"AD,AR",5,2,0,CHDgene,90,"[Atrial septal defect, Atrioventricular septal..."
ACVR2B,"ASD with minor abnormalities,VSD with minor ab...",Yes,AD,5,2,0,CHDgene,93,"[Atrial septal defect, Ventricular septal defe..."
ADAMTS10,"VSD,Obstructive lesions",Yes,AR,5,1,0,CHDgene,81794,"[Ventricular septal defect, Aortic stenosis, P..."
...,...,...,...,...,...,...,...,...,...
UBR1,"ASD with minor abnormalities,VSD with minor ab...",Yes,"AR,CH",5,3,0,CHDgene,197131,"[Atrial septal defect, Ventricular septal defe..."
WASHC5,"ASD with minor abnormalities,VSD with minor ab...",Yes,AR,5,2,0,CHDgene,9897,"[Atrial septal defect, Ventricular septal defe..."
ZEB2,"ASD with minor abnormalities,VSD with minor ab...",Yes,"AD,DN",5,3,1,CHDgene,9839,"[Atrial septal defect, Ventricular septal defe..."


In [ ]:
#save
CHDgene.to_csv('/home/jovyan/data/monogenic_disease/CHDgene_with_detailed_phenotype.csv')

# Define a list of phenotypes and search terms for those

In [80]:
# Gets list of unique CHD traits

list_of_bullet_points_for_each_gene=CHDgene['detailed_CHD_phenotype'].tolist()

# Get list of unique CHD phenotypes
li=[]
for list_of_bullets in list_of_bullet_points_for_each_gene:
    for bullet in list_of_bullets:
        li.append(bullet)
li=sorted(list(set(li)))

In [84]:
li

['Aberrant supraclavicular artery',
 'Abnormal fusion of right posterior semicircular canal',
 'Absent pulmonary valve',
 'Anomalous left coronary artery from the pulmonary artery',
 'Anomalous right subclavian artery',
 'Aortic aneurysm',
 'Aortic hypoplasia',
 'Aortic stenosis',
 'Atrial septal defect',
 'Atrioventricular septal defect',
 'Bicuspid aortic valve',
 'Coarctation of the aorta',
 'Coronary artery anomaly',
 'Dextrocardia',
 'Dilatation of the pulmonary trunk',
 'Double outlet right ventricle',
 'Double-inlet left ventricle',
 'Ductus arteriosus',
 'Dysplastic mitral valve',
 'Dysplastic pulmonary valve',
 'Dysplastic tricuspid valve',
 'Dysplastic valves',
 "Ebstein's anomaly",
 'Functional single ventricle',
 'Heterotaxy',
 'Hypoplastic left heart syndrome',
 'Hypoplastic left pulmonary artery',
 'Hypoplastic right ventricle',
 'Hypoplastic rv',
 'Interrupted aortic arch',
 'Left ventricular noncompaction',
 'Lsvc and left pulmonary artery arising from the ductus arteri

In [81]:
# Using the list of phenotypes (above), make a dictionary grouping them
phenotype_dict={
    'Heterotaxy':['eterotaxy','Dextrocardia'],
    'ASD':['ASD','trial septal','Atrial septal defect','Patent foramen ovale'],
    'VSD':['VSD','entricular septal','Ventricular septal defect'],
    'AVSD':['AVSD','trioventricular septal','Atrioventricular septal defect'],
    'Malformation of outflow tracts':['Truncus arteriosus','Malformation of outflow tracts','Fallot','fallot','Double outlet right ventricle','Pentalogy of fallot','Transposition of the great arteries','Transposition of the great vessels',],
    'Functional single ventricle':['Functional single ventricle','Double-inlet left ventricle','Hypoplastic left heart syndrome','Hypoplastic right ventricle','Hypoplastic rv'],
#    'Obstructive lesions':['Obstructive lesions','alv','stenosis'], # removed because I don't know how to define this term
    'Cardiomyopathy':['ardiomyopathy'],
    'DCM':['DCM','ilated'],
    'HCM':['HCM','ypertrophic'],
    'LVNC':['ompaction','LVNC','Left ventricular noncompaction'],
    'Venous anomaly':['Total anomalous pulmonary venous return','Pulmonary vein stenosis','Partial anomalous pulmonary venous return','Persistent left superior vena cava'],
    'Pulmonary artery':['Dilatation of the pulmonary trunk','Hypoplastic left pulmonary artery','Pulmonary artery dysplasia and hypoplasia','Pulmonary atresia','Pulmonary trunk and pulmonary artery absence'],
    'Aortic arch':['Right aortic arch','Patent ductus arteriosus','Ductus arteriosus','Coarctation of the aorta','Interrupted aortic arch','Lsvc and left pulmonary artery arising from the ductus arteriosus'],
    'Aorta':['Aberrant supraclavicular artery','Anomalous right subclavian artery','Aortic aneurysm','Aortic hypoplasia'],
    'Coronary':['Anomalous left coronary artery from the pulmonary artery','Coronary artery anomaly'],
    'VA valves':['Pulmonic stenosis','Absent pulmonary valve','Bicuspid aortic valve', 'Aortic stenosis','Dysplastic pulmonary valve','Pulmonary stenosis'],
    'AV valves':['Dysplastic mitral valve','Dysplastic tricuspid valve',"Ebstein's anomaly",'Mitral atresia','Mitral stenosis','Mitral valve dysplasia','Tricuspid atresia','Tricuspid stenosis','Tricuspid valve atresia','Tricuspid valve dysplasia']
}

In [82]:
CHDgene['detailed_CHD_phenotype_string'] = [','.join(map(str, l)) for l in CHDgene['detailed_CHD_phenotype']] # converts each value (which is a list) to a string

In [83]:
# make columns for each phenotype which indcates whether a gene is associated with that phenotype

phenotype_column='detailed_CHD_phenotype_string'

for phenotype in range(len(phenotype_dict)):
    CHDgene['phenotype_'+str(list(phenotype_dict.keys())[phenotype])]=0
    CHDgene['phenotype_'+str(list(phenotype_dict.keys())[phenotype])][CHDgene[phenotype_column].str.contains('|'.join(list(phenotype_dict.values())[phenotype]))]=1
    
CHDgene

/tmp/ipykernel_5094/2640117712.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CHDgene['phenotype_'+str(list(phenotype_dict.keys())[phenotype])][CHDgene[phenotype_column].str.contains('|'.join(list(phenotype_dict.values())[phenotype]))]=1
/tmp/ipykernel_5094/2640117712.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CHDgene['phenotype_'+str(list(phenotype_dict.keys())[phenotype])][CHDgene[phenotype_column].str.contains('|'.join(list(phenotype_dict.values())[phenotype]))]=1
/tmp/ipykernel_5094/2640117712.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cav

,CHD classification,Extra cardiac phenotype,Inheritance mode,Ranking,Supporting References,TF,source,gene_entrez_ids,detailed_CHD_phenotype,detailed_CHD_phenotype_string,...,phenotype_DCM,phenotype_HCM,phenotype_LVNC,phenotype_Venous anomaly,phenotype_Pulmonary artery,phenotype_Aortic arch,phenotype_Aorta,phenotype_Coronary,phenotype_VA valves,phenotype_AV valves
gene,,,,,,,,,,,,,,,,,,,,,
ABL1,"ASD,VSD",Yes,"AD,DN",5,2,1,CHDgene,25,"[Atrial septal defect, Ventricular septal defect]","Atrial septal defect,Ventricular septal defect",...,0,0,0,0,0,0,0,0,0,0
ACTC1,"ASD,VSD",Yes,AD,5,3,0,CHDgene,70,"[Atrial septal defect, Ventricular septal defect]","Atrial septal defect,Ventricular septal defect",...,0,0,0,0,0,0,0,0,0,0
ACVR1,"ASD,AVSD and variants,Malformation of outflow ...",No,"AD,AR",5,2,0,CHDgene,90,"[Atrial septal defect, Atrioventricular septal...","Atrial septal defect,Atrioventricular septal d...",...,0,0,0,0,0,0,0,0,0,0
ACVR2B,"ASD with minor abnormalities,VSD with minor ab...",Yes,AD,5,2,0,CHDgene,93,"[Atrial septal defect, Ventricular septal defe...","Atrial septal defect,Ventricular septal defect...",...,0,0,0,1,1,0,0,0,1,1
ADAMTS10,"VSD,Obstructive lesions",Yes,AR,5,1,0,CHDgene,81794,"[Ventricular septal defect, Aortic stenosis, P...","Ventricular septal defect,Aortic stenosis,Pulm...",...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UBR1,"ASD with minor abnormalities,VSD with minor ab...",Yes,"AR,CH",5,3,0,CHDgene,197131,"[Atrial septal defect, Ventricular septal defe...","Atrial septal defect,Ventricular septal defect...",...,0,0,0,0,0,1,0,0,0,0
WASHC5,"ASD with minor abnormalities,VSD with minor ab...",Yes,AR,5,2,0,CHDgene,9897,"[Atrial septal defect, Ventricular septal defe...","Atrial septal defect,Ventricular septal defect...",...,0,0,0,0,0,1,0,0,0,0
ZEB2,"ASD with minor abnormalities,VSD with minor ab...",Yes,"AD,DN",5,3,1,CHDgene,9839,"[Atrial septal defect, Ventricular septal defe...","Atrial septal defect,Ventricular septal defect...",...,0,0,0,0,0,1,0,0,1,0


In [85]:
CHDgene.to_csv('/home/jovyan/data/CHD_gene_with_phenotype_annotation.csv')